# 객체 탐지
- 한 이미지에서 객체와 그 경계 상자를 탐지
- 객제 탐지 알고리즘은 일반적으로 이미지를 입력으로 받고, 경계 상자와 객체 클래스 리스트를 출력
- 경계  상자에 대해 그에 대응하는 예측 클래스와 클래스의 신뢰도를 출력


In [2]:
# Bounding Box : 이미지에서 하나의 객체 전체를 포함하는 가장 작은 직사각형
# IoU : 실측값과 모델이 예측한 값이 얼마나 겹치는지를 나타내는 지표
import numpy as np
def copute_iou(pred_box, gt_box):
    x1 = np.maximum(pred_box[0], gt_box[0])
    y1 = np.maximum(pred_box[1], gt_box[1])
    x2 = np.maximum(pred_box[2], gt_box[2])
    y2 = np.maximum(pred_box[3], gt_box[3])
    
    intersetcion = np.maximum(x2 - x1, 0) * np.maximum(y2 - y1, 0)
    
    pred_box_area = (pred_box[2]- pred_box[0]*(pred_box[3]-pred_box[1]))
    gt_box_area = (gt_box[2]- gt_box[0]*(gt_box[3]-gt_box[1]))
    
    union = pred_box_area + gt_box_area - intersetcion
    
    iou = intersetcion / union
    
    return iou
    
    
   

# NMS:비최댓값 억제
- 확률이 가장 높은 상자와 겹치는 상자들을 제거하는 과정
- 최댓값을 갖지 않는 상자들을 제거

In [3]:
import numpy as np

def non_max_suppression_fast(boxes, overlap_thresh):
    if len(boxes) == 0:
        return []
    
    if boxes.dtype.kind == 'i':
        boxes = boxes.astype('float')
        
    pick = []
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    area = (x2 - x1 + 1)*(y2 - y1 + 1)
    idxs = np.argsort(y2)
    
    while len(idxs) > 0:
        last = len(idxs)-1 
        i = idxs[last] 
        pick.append(i)
        
        xx1 = np.maximum(x1[i], x1[idxs[:last]]) 
        yy1 = np.maximum(x1[i], y1[idxs[:last]]) 
        xx2 = np.maximum(x1[i], x2[idxs[:last]]) 
        yy2 = np.maximum(x1[i], y2[idxs[:last]])
        
        w = np.maximum(0, xx2 - xx1 +1)
        h = np.maximum(0, yy2 - yy1 +1)
        
        overlap = (w*h) / area[idxs[:last]]  
        
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlap_thresh)[0])))
    return boxes[pick].astype('int')   

# 모델 성능 평가

# 정밀도와 재현율 
- 일반적으로 객체 탐지 모델 평가에 사용되지는 않지만, 다른 지표를 계산하는 기본 지표 역할을 함

    - TP : 예측이 동일 클래스의 실제 상자와 일치하는지 
    - FP : 예측이 실제 상자와 일치하지 않는지
    - FN : 실제 분류값이 그와 일치하는 예측을 갖지 못하는지

# YOLO 
- 가장 빠른 객체 검출 알고리즘 중 하나
- 256 x 256 사이즈의 이미지
- 작은 크기의 물체 탐지 어려움 > 개선 중

# YOLO backbone
- backbone model 기반
- 특징 추출기라고도 불림
- yolo는 자체 맞춤 아키텍쳐 사용
- 어떤 추출기 아키텍쳐를 사용했는지에 따라 성능 달라짐


In [ ]:
!git clone 